In this project we will be applying different types of forecasting techniques on the given cpu data and analyise which one of them provides better accuracy and why.

We will be using folowing forecasting techniques:

1) Simple Moving Average

2) Weighted Moving average

3) Exponential Smoothning

4) Exponential Smoothning w/ trend

5) Moving weighted average EST

6) Linear Regression

7) Non-linear Regression (2nd order)

# Pre-Processing

Importing all the required libraries

In [1]:
import pandas as pd
import numpy as np
import cPickle
import time
import pyprind
import plotly.plotly as py
import plotly.graph_objs as go
import scipy

Linux src files

In [2]:
src_file = "/home/narsi/Documents/CSEE5690_data/VM-CPU-Stats-1-Day.csv"
#store_file = file('/home/narsi/Documents/CSEE5690_data/cpu_analysis/cpu_analytics_'+time.strftime("%Y%m%d-%H%M%S")+'.pkl', 'wb')
store_folder = "/home/narsi/Documents/CSEE5690_data/cpu_analysis/"

Windows src files

In [2]:
src_file = "D:\\course_related_data\\CSEE5690\\VM-CPU-Stats-1-Day.csv"
store_folder = "D:\\course_related_data\\CSEE5690\\"

Read Data from csv file

In [3]:
cpu_data = pd.read_csv(src_file).iloc[::-1]

In [4]:
cpu_freq = np.array(cpu_data['Value'],dtype = np.float32)
E_id_list = cpu_data.Entity.unique()
E_select = [25,60,110,225,350]

In [5]:
# [3, 5, 9, 11, 20]
trace_1 = go.Scatter(y=cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[25]].index.tolist(), dtype=np.int32))-1], marker={'size': "5"}, 
                    mode="lines", name=E_id_list[25]+' Mean'+
                     str(np.mean(cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[25]].index.tolist(), dtype=np.int32))-1]))
                    +' Varience'+ str(np.var(cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[25]].index.tolist(), dtype=np.int32))-1]))
                    )
trace_2 = go.Scatter(y=cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[60]].index.tolist(), dtype=np.int32))-1], marker={'size': "5"}, 
                    mode="lines", name=E_id_list[60]+' Mean'+
                     str(np.mean(cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[60]].index.tolist(), dtype=np.int32))-1]))
                    +' Varience'+ str(np.var(cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[60]].index.tolist(), dtype=np.int32))-1]))
                    )
trace_3 = go.Scatter(y=cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[110]].index.tolist(), dtype=np.int32))-1], marker={'size': "5"}, 
                    mode="lines", name=E_id_list[110]+' Mean'+
                     str(np.mean(cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[110]].index.tolist(), dtype=np.int32))-1]))
                    +' Varience'+ str(np.var(cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[110]].index.tolist(), dtype=np.int32))-1]))
                    )
trace_4 = go.Scatter(y=cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[225]].index.tolist(), dtype=np.int32))-1], marker={'size': "5"}, 
                    mode="lines", name=E_id_list[225]+' Mean'+
                     str(np.mean(cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[225]].index.tolist(), dtype=np.int32))-1]))
                    +' Varience'+ str(np.var(cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[225]].index.tolist(), dtype=np.int32))-1]))
                    )
trace_5 = go.Scatter(y=cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[350]].index.tolist(), dtype=np.int32))-1], marker={'size': "5"}, 
                    mode="lines", name=E_id_list[350]+' Mean'+
                     str(np.mean(cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[350]].index.tolist(), dtype=np.int32))-1]))
                    +' Varience'+ str(np.var(cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[350]].index.tolist(), dtype=np.int32))-1]))
                    )

In [6]:
data=go.Data([trace_1,trace_2,trace_3,trace_4,trace_5])
layout=go.Layout()
figure=go.Figure(data=data,layout=layout)
py.iplot(figure)

In [9]:
cpu_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146004 entries, 146003 to 0
Data columns (total 9 columns):
Value           146004 non-null int64
Timestamp       146004 non-null object
MetricId        146004 non-null object
Unit            146004 non-null object
Description     146004 non-null object
Entity          146004 non-null object
EntityId        146004 non-null object
IntervalSecs    146004 non-null int64
Instance        0 non-null float64
dtypes: float64(1), int64(2), object(6)
memory usage: 10.0+ MB


# 1) Moving Average

In [7]:
MA = pd.DataFrame(columns=['Subject','Window','TS_Score','MAD','A','F'])
bar = pyprind.ProgBar(len(E_select)*5, stream=1)
dataframe_counter = 0
for E_id in E_id_list[E_select]:
    id_list = (len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id].index.tolist(), dtype=np.int32))-1
    for window in [3, 5, 9, 11, 20]:
        x = np.empty(len(id_list) - window)
        counter = 0
        for i in id_list[window:]:
            X = cpu_freq[i - window:i]
            x[counter] = np.sum(X) * 1.0 / window
            counter += 1

        TS = np.sum(cpu_freq[id_list[window:]]-x)/(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x))
        #MA.append({'Window': window, 'Actual_Data': cpu_freq[id_list[window:]], 'Predicted': x, 'TS_Score': TS}, ignore_index=True)
        MA.loc[dataframe_counter] = [E_id,window,TS,(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x)),cpu_freq[id_list[window:]],x]
        dataframe_counter += 1
        bar.update()

MA.to_csv(store_folder+'/MA_'+time.strftime("%Y%m%d-%H%M%S")+'.csv')

0%                     100%
[#########################] | ETA: 00:00:00
Total time elapsed: 00:00:00


# 2) Weighted Moving Average

In [8]:
WMA = pd.DataFrame(columns=['Subject','Window','TS_Score','MAD','A','F'])
bar = pyprind.ProgBar(len(E_select)*5, stream=1)
dataframe_counter = 0
for E_id in E_id_list[E_select]:
    id_list = (len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id].index.tolist(), dtype=np.int32))-1
    for window in [3, 5, 9, 11, 20]:
        x = np.empty(len(id_list)-window)
        counter = 0
        weight_ = np.arange(window,dtype = np.float32)+1
        weight_ = weight_/np.sum(weight_)
        
        for i in id_list[window:]:
            x[counter] = np.sum(cpu_freq[i-window:i] * weight_)
            counter += 1
        
        TS = np.sum(cpu_freq[id_list[window:]]-x)/(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x))
        WMA.loc[dataframe_counter] = [E_id,window,TS,(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x)),cpu_freq[id_list[window:]],x]
        dataframe_counter += 1
        bar.update()
WMA.to_csv(store_folder+'/WMA_'+time.strftime("%Y%m%d-%H%M%S")+'.csv')

0%                     100%
[#########################] | ETA: 00:00:00
Total time elapsed: 00:00:00


# 3) Exponential Smoothing

In [9]:
ES = pd.DataFrame(columns=['Subject','Alpha','TS_Score','MAD','A','F'])

alpha = [0.2,0.4,0.6,0.8]
dataframe_counter = 0
bar = pyprind.ProgBar(len(alpha)*len(E_select), stream=1)
for a in alpha:
    for E_id in E_id_list[E_select]:
        id_list = (len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id].index.tolist(), dtype = np.int32))-1
        x = np.empty(len(id_list))
        x[0] = cpu_freq[id_list[0]]
        counter = 1
        for i in id_list[1:]:
            x[counter] = x[counter-1] + (a) * (cpu_freq[i-1] - x[counter-1])
            counter += 1
        
        TS = np.sum(cpu_freq[id_list]-x)/(np.sum(np.absolute(cpu_freq[id_list]-x))/len(x))
        ES.loc[dataframe_counter] = [E_id,a,TS,(np.sum(np.absolute(cpu_freq[id_list]-x))/len(x)),cpu_freq[id_list],x]
        dataframe_counter +=1
        bar.update()
ES.to_csv(store_folder+'/ES_'+time.strftime("%Y%m%d-%H%M%S")+'.csv')

0%                100%
[####################] | ETA: 00:00:00
Total time elapsed: 00:00:00


# 4) Exponential Smoothing with trend

In [10]:
EST = pd.DataFrame(columns=['Subject','Alpha','TS_Score','MAD','A','F'])
d = 0.5
alpha = [0.2,0.4,0.6,0.8]
dataframe_counter = 0
bar = pyprind.ProgBar(len(alpha)*len(E_select), stream=1)
for a in alpha:
    for E_id in E_id_list[E_select]:
        id_list = (len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id].index.tolist(), dtype = np.int32))-1
        x = np.empty(len(id_list))
        x[0] = cpu_freq[id_list[0]]
        T = 0.0
        F = x[0]
        counter = 1
        for i in id_list[1:]:
            F = x[counter - 1] + a * (cpu_freq[i-1] - x[counter-1])
            T = T + d * (F - x[counter-1])
            x[counter] = F + T
            counter += 1

        TS = np.sum(cpu_freq[id_list]-x)/(np.sum(np.absolute(cpu_freq[id_list]-x))/len(x))
        EST.loc[dataframe_counter] = [E_id,a,TS,(np.sum(np.absolute(cpu_freq[id_list]-x))/len(x)),cpu_freq[id_list[window:]],x]
        dataframe_counter += 1
        bar.update()
EST.to_csv(store_folder+'/EST_'+time.strftime("%Y%m%d-%H%M%S")+'.csv')

0%                100%
[####################] | ETA: 00:00:00
Total time elapsed: 00:00:00


# 5) Windowed Exponential Smoothing with trend V1

In [11]:
WEST = pd.DataFrame(columns=['Subject','Alpha','Window','TS_Score','A','F'])
d = 0.5
alpha = [0.2,0.4,0.6,0.8]
bar = pyprind.ProgBar(5*4*len(E_select), stream=1)
dataframe_counter = 0
for a in alpha:
    for E_id in E_id_list[E_select]:
        id_list = (len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id].index.tolist(), dtype = np.int32))-1
        for window in [3, 5, 9, 11, 20]:
            x = np.empty(len(id_list))
            weight_ = np.arange(window,dtype = np.float32)+1
            weight_ = weight_/np.sum(weight_)
            x[0:window] = cpu_freq[id_list[0:window]]
            T = 0.0
            F = x[window-1]
            counter = window
            for i in id_list[window:]:
                F = x[counter - 1] + a * (np.sum((cpu_freq[i-window:i] - x[counter-window:counter])*weight_))
                T = T + d * (F - x[counter-1])
                x[counter] = F + T
                if x[counter] <= 0:
                    x[counter] = np.sum(cpu_freq[i-window:i]*weight_)
                counter += 1

            TS = np.sum(cpu_freq[id_list[window:]]-x[window:])/(np.sum(np.absolute(cpu_freq[id_list[window:]]-x[window:]))/len(x))
            WEST.loc[dataframe_counter] = [E_id,a,window,TS,cpu_freq[id_list[window:]],x[window:]]
            dataframe_counter += 1
            bar.update()
WEST.to_csv(store_folder+'/WEST_'+time.strftime("%Y%m%d-%H%M%S")+'.csv')

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


# 6) Moving Window Linear Regression

In [12]:
PL = pd.DataFrame(columns=['Subject','Window','TS_Score','MAD','A','F'])
bar = pyprind.ProgBar(5*len(E_select), stream=1)
dataframe_counter = 0
for E_id in E_id_list[E_select]:
    id_list = (len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id].index.tolist(), dtype=np.int32))-1
    for window in [3, 5, 9, 11, 20]:
        x = np.empty(len(id_list)-window)
        t = np.arange(window)
        weight_ = np.arange(window,dtype = np.float32)+1
        weight_ = weight_/np.sum(weight_)
        counter = 0
        for i in id_list[window:]:
            (ar,br) = scipy.polyfit(t,cpu_freq[i-window:i],1)
            x[counter] = scipy.polyval([ar,br],window)
            counter += 1
            
        TS = np.sum(cpu_freq[id_list[window:]]-x)/(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x))
        PL.loc[dataframe_counter] = [E_id,window,TS,(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x)),cpu_freq[id_list[window:]],x]
        dataframe_counter += 1
        bar.update()
PL.to_csv(store_folder+'/PL_'+time.strftime("%Y%m%d-%H%M%S")+'.csv')

0%                     100%
[#########################] | ETA: 00:00:00
Total time elapsed: 00:00:01


# 7) Moving Window 2nd order Regression

In [13]:
PL2 = pd.DataFrame(columns=['Subject','Window','TS_Score','MAD','A','F'])
bar = pyprind.ProgBar(5*len(E_select), stream=1)
dataframe_counter = 0
for E_id in E_id_list[E_select]:
    id_list = (len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id].index.tolist(), dtype=np.int32))-1
    for window in [3, 5, 9, 11, 20]:
        x = np.empty(len(id_list)-window)
        t = np.arange(window)
        weight_ = np.arange(window,dtype = np.float32)+1
        weight_ = weight_/np.sum(weight_)
        counter = 0
        for i in id_list[window:]:
            z = np.polyfit(t,cpu_freq[i-window:i],2)
            p = np.poly1d(z)
            x[counter] = p(window)
            counter += 1
            
        TS = np.sum(cpu_freq[id_list[window:]]-x)/(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x))
        PL2.loc[dataframe_counter] = [E_id,window,TS,(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x)),cpu_freq[id_list[window:]],x]
        dataframe_counter += 1
        bar.update()
PL2.to_csv(store_folder+'/PL2_'+time.strftime("%Y%m%d-%H%M%S")+'.csv')

0%                     100%
[#########################] | ETA: 00:00:00
Total time elapsed: 00:00:00


# Plotting different methods results for "kc-pd-sql01" VM

In [14]:
[original_signal] = PL[(PL['Subject'] == 'kc-pd-sql01') & (PL['Window'] ==  20)].A.tolist()
x_axis = np.arange(1,len(original_signal)+1)
[MA_20W_signal] = MA[(MA['Subject'] == 'kc-pd-sql01') & (MA['Window'] ==  20)].F.tolist()
[WMA_20W_signal] = WMA[(WMA['Subject'] == 'kc-pd-sql01') & (WMA['Window'] ==  20)].F.tolist()
[ES_A8_signal] = ES[(ES['Subject'] == 'kc-pd-sql01') & (ES['Alpha'] ==  0.8)].F.tolist()
ES_A8_signal = ES_A8_signal[20:]
[EST_A8_signal] = EST[(EST['Subject'] == 'kc-pd-sql01') & (EST['Alpha'] ==  0.8)].F.tolist()
EST_A8_signal = EST_A8_signal[20:]
[WEST_3W_A8_signal] = WEST[(WEST['Subject'] == 'kc-pd-sql01') & (WEST['Alpha'] ==  0.8) & (WEST['Window'] ==  20)].F.tolist()
[PL_20W_signal] = PL[(PL['Subject'] == 'kc-pd-sql01') & (PL['Window'] ==  20)].F.tolist()
[PL2_20W_signal] = PL2[(PL2['Subject'] == 'kc-pd-sql01') & (PL2['Window'] ==  20)].F.tolist()
#trace_orignal = go.Scatter(x = range(1,len()))

In [17]:
# [3, 5, 9, 11, 20]
trace_o = go.Scatter( x = x_axis,y=original_signal, marker={'size': "5"}, 
                    mode="lines", name='Original Signal')
trace_est = go.Scatter( x = x_axis,y=EST_A8_signal, marker={'size': "5"}, 
                    mode="markers", name='Exponential Smoothening w/ Trend')
trace_pl = go.Scatter( x = x_axis,y=PL_20W_signal, marker={'size': "5"}, 
                    mode="lines", name='Moving Window 2nd order Regression Linear', line={'dash':'dash'})
trace_pl2 = go.Scatter( x = x_axis,y=PL2_20W_signal, marker={'size': "5"}, line={'dash':'dot'}, 
                    mode="markers+lines", name='Moving Window 2nd order Regression 2nd order polynomial')


In [18]:
data=go.Data([trace_o,trace_est,trace_pl,trace_pl2])
layout=go.Layout()
figure=go.Figure(data=data,layout=layout)
py.iplot(figure)